# Design filter mixer design for adc_receiver_v0

In [ ]:
import numpy as np
import scipy
import scipy.signal
import matplotlib.pyplot as plt
%matplotlib widget
#%matplotlib qt

## Parameters of the dec16_fir filter

- 16 samples in //
- 4 taps
- coefficients represented as Q(16, 15)

In [ ]:
n_inputs = 4      # Number of Simultaneous Inputs (2^?)
decimation = 2**n_inputs
TotalTaps = 4     # Total Number of Taps
PFBSize = 5       # We want the same here
alltaps = TotalTaps * 2**PFBSize
fwidth = 1

coef_w = 16
coef_dp = 15

fs = 3700
F_cut = fs/decimation/2

In [ ]:
dec16_fir_coeffs = scipy.signal.firwin(alltaps, F_cut , width=None, window='hamming', pass_zero=True, scale=True, nyq=None, fs=fs)
dec16_fir_coeffs /= np.max(dec16_fir_coeffs)
#windowval = scipy.signal.windows.hamming(alltaps, sym=True)
#coeffs = windowval * np.sinc(fwidth * ((np.arange(alltaps) + 0.5)/(2**PFBSize)-TotalTaps/2))

In [ ]:
def aliased_freq(f_signal, f_sample):
    n = (f_signal / float(f_sample)).round()
    f_alias = abs(f_sample * n - f_signal)
    return f_alias


In [ ]:
def plot_coeff(coeffs, fs, decimation):
    fig, axs = plt.subplots(ncols=3, figsize=(15,5))
    axs[0].plot(coeffs)

    w, h = scipy.signal.freqz(coeffs, worN=np.linspace(-fs/2,fs/2,2048), whole=True, fs=fs)
    H = 20*np.log10(abs(h))
    H -= np.max(H)
    
    axs[1].plot(w, H)
    axs[1].set_xlim((-fs/decimation*1.2, fs/decimation*1.2))
    axs[1].set_ylim((-75, 1))
    
    wa = aliased_freq(w, fs/decimation)
    axs[1].plot(wa, H)
    axs[1].plot(-wa, H)
    axs[1].axvline(-fs/decimation/2, color='r')
    axs[1].axvline(fs/decimation/2, color='r')
    axs[1].axvline(-fs/decimation/4, color='k')
    axs[1].axvline(fs/decimation/4, color='k')
  
    
    dF0 = fs / decimation
    for ch in range(0, 16):
        axs[2].plot(w+ch*dF0, H, '-')
    
    axs[2].set_xlim((-fs/2, fs/2))
    axs[2].set_ylim((-75, 1))


plot_coeff(dec16_fir_coeffs, fs, decimation)

In [ ]:
dec16_fir_coeffs.tofile('dec_fir_coefs.txt', sep=', ')
print('[' + ', '.join( str(c) for c in list(dec16_fir_coeffs)) + ']')
print(dec16_fir_coeffs.shape)

In [ ]:
def quantize(x, w, pt, mode='round'):
    max_v = (2**(w-1) - 1) / 2**pt
    min_v = (-2**(w-1))    / 2**pt
    #print(f"[{min_v},{max_v}]")

    # normalize
    max_x = np.max(x)
    if max_x > max_v:
        x = x / max_x * max_v

    min_x = np.min(x)
    if min_x < min_v:
        x = x / min_x * min_v

    # quantize
    y = x * 2**pt
    if mode == 'round':
        y = np.round(y)
    elif mode == 'trunc':
        y = np.floor(y)
    return y / 2**pt

# x = np.arange(-1024, 1023)/1024
# plt.figure()
# plt.plot(x, quantize(x, 6), '.')
# plt.plot(x, quantize(x, 8), '.')
# plt.plot(x, quantize(x, 10), '.')


In [ ]:
Q = (coef_w, coef_dp)
# Q = (6, 5)
dec16_fir_coeffs_q = quantize(dec16_fir_coeffs, Q[0], Q[1])
plot_coeff(dec16_fir_coeffs_q, fs, decimation)


In [ ]:
dec16_fir_coeffs_q.tofile(f'dec_fir_coefs_Q{Q[0]}.{Q[1]}.txt', sep=', ')
print('[' + ', '.join( str(c) for c in list(dec16_fir_coeffs_q)) + ']')
print(dec16_fir_coeffs_q.shape)

In [ ]:
# ~~[Filter Design with Parks-McClellan Remez]~~
N = 10  # Filter order
# Filter symetric around 0.25 (where .5 is pi or Fs/2)
transition_band = 0.25
bands = np.array([0., .25-transition_band/2, .25+transition_band/2, .5])
h2 = scipy.signal.remez(N+1, bands, [1,0], [1,1])
h2[abs(h2) <= 1e-4] = 0.
h2 /= np.max(h2)

plot_coeff(h2, fs/16, 2)

In [ ]:
# ~~[Filter Design with Parks-McClellan Remez]~~
N = 40  # Filter order
# Filter symetric around 0.25 (where .5 is pi or Fs/2)
transition_band = 0.07
bands = np.array([0., .25-transition_band/2, .25+transition_band/2, .5])
h0 = scipy.signal.remez(N+1, bands, [1,0], [1,1])
h0[abs(h0) <= 1e-4] = 0.
h0 /= np.max(h0)

plot_coeff(h0, fs/16, 2)

In [ ]:
def aliased_freq(f_signal, f_sample):
    neg = f_signal < 0
    n = (f_signal / float(f_sample)).round()
    f_alias = abs(f_sample * n - f_signal)
    f_alias[neg] *= -1
    return f_alias

In [ ]:
def plot_receiver_gain(Galactic_mode):
    Npts = 2048*16
    fs = 3700
    f = np.linspace(0, fs, Npts, endpoint=False)
    f = np.fft.fftshift(f)
    f[f>=fs/2] = f[f>=fs/2]-3700
    decimation = 1

    dec16_fir_coeffs_q = quantize(dec16_fir_coeffs, *Q)
    h2_q               = quantize(              h2, *Q)
    h0_q               = quantize(              h0, *Q)

    filters = [("dec16", dec16_fir_coeffs_q, 16, None, True),
            (   "h2",               h2_q,  2, None, Galactic_mode),
            (   "h1",               h2_q,  2, None, Galactic_mode),
            (   "h0",               h0_q,  2, None, True),
            ]

    for filter_idx in range(len(filters)):
        filter_name, filter_coeff, filter_decimation, filter_gain, active = filters[filter_idx]
        if active:
            _, filter_gain = scipy.signal.freqz(filter_coeff, worN=f, fs=fs/decimation)
            filter_gain /= np.max(filter_gain)
            decimation *= filter_decimation
            filters[filter_idx] = (filter_name, filter_coeff, decimation, filter_gain, active)
    
    
    
    gain = np.ones_like(filter_gain)
    
    fig, axs = plt.subplots(nrows=len(filters) + 1, ncols=3,
                            sharex='col', sharey=True,
                            figsize=(15,10))
    for filter_idx in range(len(filters)):
        filter_name, filter_coeff, decimation, filter_gain, active = filters[filter_idx]
        if active:
            gain *= filter_gain
            axs[filter_idx][0].plot(f, 20*np.log10(abs(filter_gain)))
            axs[filter_idx][1].plot(aliased_freq(f, fs/decimation), 20*np.log10(abs(filter_gain)))
            axs[filter_idx][2].plot(aliased_freq(f, fs/decimation), 20*np.log10(abs(filter_gain)))
    
     
    
    axs[-1][0].plot(f, 20*np.log10(abs(gain)))
    axs[-1][1].plot(aliased_freq(f, fs/decimation), 20*np.log10(abs(gain)))
    axs[-1][2].plot(aliased_freq(f, fs/decimation), 20*np.log10(abs(gain)))
    
    axs[-1][1].set_ylim((-100, 3))
    if Galactic_mode:
        axs[-1][2].set_xlim((-20, 20))
    else:
        axs[-1][2].set_xlim((-80, 80))
    
    _=list(ax[0].set_ylabel("dB") for ax in axs)
    _=list(ax.set_xlabel("Freq (MHz)") for ax in axs[-1])
    
    
plot_receiver_gain(Galactic_mode = True)  #  28MHz BW
plot_receiver_gain(Galactic_mode = False)  # 112MHz BW


In [ ]:
for filter_idx in range(len(filters)):
    filter_name, filter_coeff, filter_decimation, filter_gain, active = filters[filter_idx]
    filter_coeff.tofile(f'{filter_name}_Q{Q[0]}.{Q[1]}.txt', sep=', ')
    print('[' + ', '.join( str(c) for c in list(filter_coeff)) + ']')
    print(filter_coeff.shape)
